<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

from tqdm import tqdm

# cifar10 dataset

In [ ]:
cifar10 = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images/255
test_images = test_images/255

In [ ]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [ ]:
train_labels = dummy_labels(train_labels)
test_labels = dummy_labels(test_labels)

In [ ]:
# 1000 labeled, 49000 unlabeled
indx = random.sample(range(train_labels.shape[0]),train_labels.shape[0])

lbl_train_images = train_images[indx[:1000]]
ubl_train_images = train_images[indx[1000:]]

lbl_train_labels = train_labels[indx[:1000]]
ubl_train_labels = train_labels[indx[1000:]]

# valids1 =  train_images[indx[800:1000]]
# valids2 =  train_labels[indx[800:1000]]

# pseudo labeling

### Mixup Augmentation


In [ ]:
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

def mixup (size, data, alpha = 0.2):
  image, label = data
  L = sample_beta_distribution(size, alpha, alpha)
  XL = tf.reshape(L, (size, 1, 1, 1))
  YL = tf.reshape(L, (size, 1))
  IND1 = np.random.choice(len(label), size)
  IND2 = np.random.choice(len(label), size)
  newimage = XL*image[IND1] + (1-XL)*image[IND2]
  newlabel = YL*label[IND1] + (1-YL)*label[IND2]
  return (newimage, newlabel)

### 스케줄러

In [ ]:
class SGDR(Callback):

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

### main

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  conv1a = Conv2D(128, (3,3), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(128, (3,3), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(128, (3,3), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.5)

  conv2a = Conv2D(256, (3,3), padding = 'same')
  bn2a = BatchNormalization()
  conv2b = Conv2D(256, (3,3), padding = 'same')
  bn2b = BatchNormalization()
  conv2c = Conv2D(256, (3,3), padding = 'same')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2)
  MCdrop2 = PermaDropout(0.5)

  conv3a = Conv2D(512, (3,3))
  bn3a = BatchNormalization()
  conv3b = Conv2D(256, (1,1))
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, (1,1))
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D(6, 2)

  fc = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      conv1a, bn1a, activ,
                      conv1b, bn1b, activ,
                      conv1c, bn1c, activ,
                      pl1, MCdrop1,

                      conv2a, bn2a, activ,
                      conv2b, bn2b, activ,
                      conv2c, bn2c, activ,
                      pl2, MCdrop2,

                      conv3a, bn3a, activ,
                      conv3b, bn3b, activ,
                      conv3c, bn3c, activ,
                      pl3, Flatten(),
                      
                      fc
                      ])
  
  return model

def compile_cnn_13(model):

  opt = keras.optimizers.SGD(0.03, momentum=0.9)

  model.compile(
    optimizer = opt,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  X = lbl_train_images
  y = lbl_train_labels

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러
  
#   size = len(y) * 7
#   newimage, newlabel = mixup(size, (X, y))
#   X = np.concatenate((X, newimage))
#   y = np.concatenate((y, newlabel))
#   del newimage, newlabel

  model.fit(
      x=X,
      y=y,
      epochs=Epoch,
      verbose=0,
#       validation_data = (valids1, valids2),
      batch_size=Batch,
#       callbacks=[sgdr, early_stopper]
      callbacks=[sgdr]
  )
    
  model_test_eval(model, test_images, test_labels)
  T = 1

  for predsamples in (range(10)):
    if predsamples == 0 :
      predictions = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

def model_test_eval(model, test_images, test_labels):
  T = 1
  pred = np.array(tf.nn.softmax(model.predict(test_images)/T))
  for i in range(1,10):
    pred += np.array(tf.nn.softmax(model.predict(test_images)))
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  acc = sum(acc)/len(acc)
  print("test set 성능 : " + str(acc))

In [ ]:
def label_selecting():
  K_conf = 0.9
  K_uncert = 0.05

  pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
  conf = np.max(np.mean(predictions, axis=0), axis=1)
  uncert = np.std(predictions, axis=0)
  uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])

  select_pseudo = (1*(conf > K_conf)) * (1*(uncert < K_uncert))

  labels = []
  for i in pseudo:
    temp = [0,0,0,0,0,0,0,0,0,0]
    temp[i] = 1
    labels.append(temp)
  pseudo = np.array(labels)
#   pseudo = np.mean(predictions, axis=0)

  lbl_idx = []
  ubl_idx = []
  k = 0
  for i in select_pseudo:
    if i == 1:
      lbl_idx.append(k)
    if i == 0:
      ubl_idx.append(k)
    k += 1

    
  ubl_append = ubl_train_images[lbl_idx]
  pseudo_append = pseudo[lbl_idx]
    
  if itr < 5:
      try: 
        numsamples = np.min(list(Counter(np.argmax(pseudo_append, axis=1)).values()))
      except:
        numsamples = 0
      multlabel = np.argmax(pseudo_append, axis=1)
      sufindx = random.sample(range(len(multlabel)), len(multlabel))

      idxcounter = [0,0,0,0,0,0,0,0,0,0]
      idxsample = []

      for i in sufindx:
#         if idxcounter[multlabel[i]] < numsamples+25:
        if idxcounter[multlabel[i]] < 250:
          idxcounter[multlabel[i]] += 1
          idxsample.append(i)
      
      image1 = np.concatenate((lbl_train_images, ubl_append[idxsample]))
      label1 = np.concatenate((lbl_train_labels, pseudo_append[idxsample]))
      image2 = np.concatenate((ubl_train_images[ubl_idx], ubl_append[np.delete(list(range(len(ubl_append))), idxsample)]))
  
  else:
      image1 = np.concatenate((lbl_train_images, ubl_append))
      label1 = np.concatenate((lbl_train_labels, pseudo_append))
      image2 = ubl_train_images[ubl_idx]

  return image1, label1, image2

In [ ]:
import time
start = time.time()

for itr in range(20):
  model = cnn_13()
  print(Counter(np.argmax(lbl_train_labels, axis=1)))
  predictions = fit_and_labeling_cnn_13(1000, 128)
  lbl_train_images, lbl_train_labels, ubl_train_images = label_selecting()
  del predictions
#   teacher_model = model
  gc.collect()

print("time :", time.time() - start)
# CNN-13(update) + mixup augmentation(x4) + itr10balancing < 0.64
# CNN-13(update) + mixup augmentation(x8) + itr5balancing < 0.68?

Counter({6: 108, 3: 105, 5: 104, 9: 103, 0: 101, 7: 100, 2: 100, 4: 95, 8: 94, 1: 90})
test set 성능 : 0.5441
Counter({6: 358, 3: 355, 5: 354, 9: 353, 0: 351, 7: 350, 2: 350, 4: 345, 8: 344, 1: 340})
test set 성능 : 0.5751
Counter({6: 608, 3: 605, 5: 604, 9: 603, 0: 601, 7: 600, 2: 600, 4: 595, 8: 594, 1: 590})
test set 성능 : 0.5927
Counter({6: 858, 3: 855, 5: 854, 9: 853, 0: 851, 7: 850, 2: 850, 4: 845, 8: 844, 1: 840})
test set 성능 : 0.5967
Counter({6: 1108, 3: 1105, 5: 1104, 9: 1103, 0: 1101, 7: 1100, 2: 1100, 4: 1095, 8: 1094, 1: 1090})
test set 성능 : 0.6032
Counter({6: 1358, 3: 1355, 5: 1354, 9: 1353, 0: 1351, 7: 1350, 2: 1350, 4: 1345, 8: 1344, 1: 1340})
test set 성능 : 0.6068
Counter({1: 4384, 6: 4258, 8: 3802, 9: 3715, 3: 3643, 0: 3487, 5: 3378, 7: 3314, 2: 3209, 4: 3136})
test set 성능 : 0.6145
Counter({1: 4832, 6: 4743, 8: 4220, 3: 4205, 9: 4145, 5: 3968, 0: 3826, 2: 3770, 7: 3756, 4: 3507})
test set 성능 : 0.6234
Counter({6: 5039, 1: 5000, 3: 4466, 8: 4466, 9: 4358, 5: 4243, 2: 4051, 0: 